# A pretty report with a little wisdom

> A clever person solves a problem. A wise person avoids it.


In [ ]:
# Define input parameters
# Purchase scenario
purchase_price = 300000  # House purchase price
down_payment_rate = 0.20  # 20% down payment
mortgage_rate = 0.03  # Annual mortgage rate
mortgage_years = 25  # Mortgage duration
purchase_fees_rate = 0.04  # Notary + taxes (simplified)
agency_fees_rate = 0.03  # Agency fees for purchase
maintenance_rate = 0.015  # Annual maintenance (1.5% of house value)
condo_fees_landlord = 150  # Monthly condominium fees

# Rent scenario
monthly_rent = 1200  # Base monthly rent
deposit_months = 3  # Number of months for deposit
rent_agency_fees_months = 1  # Agency fees in months of rent
condo_fees_tenant = 75

# Common expenses (monthly)
utilities = 200  # Monthly utilities estimate

# Economic factors
inflation_rate = 0.02  # Annual inflation rate
house_appreciation_rate = 0.01  # Annual house value appreciation
investment_return_rate = 0.05  # Return rate on invested capital

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px


def calculate_mortgage_payment(principal, rate, years):
    """Calculate monthly mortgage payment"""
    monthly_rate = rate / 12
    n_payments = years * 12
    return (
        principal
        * (monthly_rate * (1 + monthly_rate) ** n_payments)
        / ((1 + monthly_rate) ** n_payments - 1)
    )


def compound_monthly(amount, rate, months):
    """Compound amount monthly at given annual rate"""
    return amount * (1 + rate / 12) ** months

In [ ]:
# Calculate scenarios
months = mortgage_years * 12
timeline = np.arange(months + 1)

# Buy scenario
mortgage_amount = purchase_price * (1 - down_payment_rate)
initial_costs = purchase_price * (
    down_payment_rate + purchase_fees_rate + agency_fees_rate
)
monthly_mortgage = calculate_mortgage_payment(
    mortgage_amount, mortgage_rate, mortgage_years
)
monthly_costs_buy = (
    condo_fees_landlord + utilities + purchase_price * maintenance_rate / 12
)

# Rent scenario
initial_costs_rent = monthly_rent * (deposit_months + rent_agency_fees_months)
monthly_costs_rent = monthly_rent + condo_fees_landlord + utilities

# Calculate net worth over time
results = pd.DataFrame(index=timeline)
results["month"] = timeline

# Buy scenario net worth
house_value = [
    purchase_price * (1 + house_appreciation_rate) ** (year / 12) for year in timeline
]
# La seconda parte dell'equazione è la sommatoria delle rate mensili pagate fino a quel mese
# con il tasso di interesse applicato, come se fossero investite a un tasso di interesse
mortgage_balance = [
    mortgage_amount * (1 + mortgage_rate) ** (month_index / 12)
    - monthly_mortgage
    * sum((1 + mortgage_rate) ** (month / 12) for month in range(month_index))
    for month_index in timeline
]
investment_buy = [
    -initial_costs
    * (1 + investment_return_rate)
    ** (year / 12)  # Money I could have invested plus interests (oppotunity cost)
    - (monthly_mortgage + monthly_costs_buy)
    * sum(
        (1 + investment_return_rate) ** ((year * 12 - month) / 12)
        for month in range(int(year * 12))
    )
    for year in timeline
]

results["buy_networth"] = (
    np.array(house_value) - np.array(mortgage_balance) + np.array(investment_buy)
)

# Rent scenario net worth
investment_rent = [
    -initial_costs_rent * (1 + investment_return_rate) ** (year / 12)
    - monthly_costs_rent
    * sum(
        (1 + investment_return_rate) ** ((year * 12 - month) / 12)
        for month in range(int(year * 12))
    )
    for year in timeline / 12
]

results["rent_networth"] = investment_rent

In [ ]:
# Visualize results
fig = px.line(
    results,
    x="month",
    y=["buy_networth", "rent_networth"],
    title="Net Worth Comparison: Buy vs Rent",
    labels={"month": "Months", "value": "Net Worth (€)", "variable": "Scenario"},
)
fig.show()

# Print final values
print(f"Final net worth after {mortgage_years} years:")
print(f"Buy scenario: {results.buy_networth.iloc[-1]:,.2f}€")
print(f"Rent scenario: {results.rent_networth.iloc[-1]:,.2f}€")